<a href="https://colab.research.google.com/github/celefigs/collaborative_project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt

import nltk
import re

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

from wordcloud import WordCloud

# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# # Crear un pipeline de procesamiento y clasificación con RandomForest
# pipeline_rf = Pipeline([
#     ('vectorizer', TfidfVectorizer()),  # O CountVectorizer()
#     ('classifier', RandomForestClassifier())
# ])


Logistic Regression

In [ ]:
# from sklearn.linear_model import LogisticRegression

# # Crear un pipeline de procesamiento y clasificación con Logistic Regression
# pipeline_lr = Pipeline([
#     ('vectorizer', TfidfVectorizer()),  # O CountVectorizer()
#     ('classifier', LogisticRegression())
# ])

In [4]:
# # Use a pipeline??
# from transformers import pipeline
# import torch

# pipe = pipeline("text-classification", model="juliensimon/reviews-sentiment-analysis")



In [5]:
# # Load model directly -> consultar Filipa
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("juliensimon/reviews-sentiment-analysis")
# model = AutoModelForSequenceClassification.from_pretrained("juliensimon/reviews-sentiment-analysis")

In [6]:
# torch

In [7]:
# from transformers import pipeline

# classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

# sentences = ["I am feeling bit weird today"]

# model_outputs = classifier(sentences)
# print(model_outputs[0])
# # produces a list of dicts for each of the labels


In [30]:
file_path = '../data/final_data.csv'

review_df=pd.read_csv(file_path, encoding='utf-8')

review_df.head()

,Unnamed: 0,provided_drug_name,condition,review,rating,date,usefulCount,matched_generic_name,matched_type
0,0,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,NaN,NaN
1,1,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,NaN,NaN
2,2,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,7-Mar-17,5,levonorgestrel,emergency
3,3,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,ethinyl estradiol / norethindrone,standard
4,4,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10,etonogestrel,standard


In [31]:
filtered_df = review_df[review_df['condition'] == 'Birth Control']
filtered_df.head()

,Unnamed: 0,provided_drug_name,condition,review,rating,date,usefulCount,matched_generic_name,matched_type
0,0,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,NaN,NaN
1,1,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,NaN,NaN
3,3,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,ethinyl estradiol / norethindrone,standard
4,4,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10,etonogestrel,standard
5,5,Etonogestrel,Birth Control,"""Nexplanon does its job. I can have worry free...",9,11-Aug-14,11,etonogestrel,standard


In [33]:
filtered_df = filtered_df.drop(['Unnamed: 0', 'condition', 'date', 'usefulCount', 'matched_generic_name', 'matched_type'], axis=1)

filtered_df.head()

,provided_drug_name,review,rating
0,Lybrel,"""I used to take another oral contraceptive, wh...",5
1,Ortho Evra,"""This is my first time using any form of birth...",8
3,Ethinyl estradiol / levonorgestrel,"""I had been on the pill for many years. When m...",8
4,Nexplanon,"""Started Nexplanon 2 months ago because I have...",3
5,Etonogestrel,"""Nexplanon does its job. I can have worry free...",9


In [34]:
filtered_df.rename(columns= {"provided_drug_name":"drugname"}, inplace= True)

In [35]:
#label  pos and neg

def label_rating(rating):
    if rating <= 5:
        return 'Negative'
    else:
        return 'Positive'

In [36]:
filtered_df['label'] = filtered_df['rating'].apply(label_rating)

In [37]:
filtered_df.head()

,drugname,review,rating,label
0,Lybrel,"""I used to take another oral contraceptive, wh...",5,Negative
1,Ortho Evra,"""This is my first time using any form of birth...",8,Positive
3,Ethinyl estradiol / levonorgestrel,"""I had been on the pill for many years. When m...",8,Positive
4,Nexplanon,"""Started Nexplanon 2 months ago because I have...",3,Negative
5,Etonogestrel,"""Nexplanon does its job. I can have worry free...",9,Positive


In [54]:
filtered_df.isnull().sum()

drugname    0
review      0
rating      0
label       0
dtype: int64

In [38]:
filtered_df['label'].value_counts()

label
Positive    16537
Negative    12251
Name: count, dtype: int64

In [17]:
# 57% of the ratings are "Positive" and  43% are "Negative".

Sanity check ____________________________________________________________________________________________________________________________________________________________________________

In [47]:
start = text.find("period lasted for two weeks and now it&#039;s the end")
end = start + len("period lasted for two weeks and now it&#039;s the end")
print(f"Start position: {start}")
print(f"End position: {end}")

Start position: 521
End position: 574


In [52]:
text = filtered_df['review'][0][521:574]
print(text)

period lasted for two weeks and now it&#039;s the end


In [53]:
# sanity check - DELETE
preprocess_review(text)

['period', 'last', 'two', 'week', 'end']

____________________________________________________________________________________________________________________________________________________________________________

In [22]:
# #Preprocess function for 'drugname' column

# def preprocess_drugname(drugname):

#     #remove special characters
#     drugname = re.sub(REPLACE_NO_SPACE, "", drugname)
#     drugname= re.sub(REPLACE_WITH_SPACE, " ", drugname)

#     # Remove leading/trailing whitespace
#     drugname = drugname.strip()

#     # Remove double spaces
#     drugname = re.sub(r'\s+', ' ', drugname)

#     return drugname

In [69]:
pd.set_option('display.max_colwidth', None)

# Splitting the dataset

I learn my vocabulary , if i dont have a tr
Can I actually build a model

In [25]:
from sklearn.model_selection import train_test_split

# X = independent variable = review,  y = target variable (dependent) = rating

X = review_df['review']
y= review_df['label']

# why we are doing these? video: 1:20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [56]:
print("DataFrame dimensions:", review_df.shape)
print(f"Train dimensions: {X_train.shape, y_train.shape}")  
print(f"Test dimensions: {X_test.shape, y_test.shape}")  

DataFrame dimensions: (31251, 3)
Train dimensions: ((23030,), (23030,))
Test dimensions: ((5758,), (5758,))


In [59]:
#Check out target distribution
print(y_train.value_counts())
print(y_test.value_counts())

#both has the same proportion, 74%

label
Positive    13232
Negative     9798
Name: count, dtype: int64
label
Positive    3305
Negative    2453
Name: count, dtype: int64


In [70]:
X_train.head()

26256                                                     "I switched to this pill after using Sprintec for years and had instant weight gain. I gained 20 pounds in the first three months. No noticeable changes in mood, sex drive, or acne. My skin stayed very clear. I also stopped having a period completely and although it was wonderful I could not get used to it. The pregnancy panics along with the weight gain forced me to stop taking these pills."
22909                                         "I&#039;ve had it for 2 weeks so far no side effects yay!!! I don&#039;t get why people keep it in for years if it has crazy side effects like weight gain, suicide etc? The moment I gain 5 lbs I&#039;m getting it out. I have a very very regular workout and diet schd. The only pills that worked for me we&#039;re Yasmin and mini pill but I kept forgetting to take it after my sons birth hence the implanon."
17874                                                                                       

# Pre-process text

In [79]:
review = X_train[1]
review

'"This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch"'

In [81]:
# Def RegEx

REPLACE_NO_SPACE = re.compile(r"[.;:!#\'?,\"()\[\]&]|\d+")  # delete punctuation and numbers
REPLACE_WITH_SPACE = re.compile(r"(<br\s*/><br\s*/>)|(\-)|(\/)")

# We want to remove everything that is not a proper character. RegEx for non-alphanumerical characters.


In [82]:
#Preprocess function for "review" column

stop_words = set(stopwords.words('english'))

def preprocess_review(reviews):

    # Remove special characters
    reviews = re.sub(REPLACE_NO_SPACE, "", reviews)  #aqui tambien podemos usar RegexpTokenizer()
    reviews = re.sub(REPLACE_WITH_SPACE, " ", reviews)

    tokens = word_tokenize(reviews)

    # Lowercase and lemmatize
    lemmatizer = WordNetLemmatizer()

    lemmas = [lemmatizer.lemmatize(token.lower(), pos='v') for token in tokens if token.lower() not in stop_words]
    lemmas = [lemmatizer.lemmatize(token.lower(), pos='n') for token in lemmas if token.lower() not in stop_words]


    return lemmas

In [83]:
#What is my weighting scheme? in this case, 

vectoriser = TfidfVectorizer(analyzer=preprocess_review)

X_train_tfidf = vectoriser.fit_transform(X_train)

In [84]:
# DataFrame dimensions: (31251, 3)
# Train dimensions: ((23030,), (23030,))
# Test dimensions: ((5758,), (5758,))

X_train_tfidf.shape # (num_1, num_2) num_1: rows, documents, num_2: unigrams, tokens

(23030, 11644)

In [85]:
#represent test documents in the Vector Space we've learned with the train documents
X_test_tfidf = vectoriser.transform(X_test)

In [86]:
X_test_tfidf.shape 


(5758, 11644)

# Train a Naive Bayes classifier

¡Perfecto! Aquí están los pasos que deberías seguir sin incluir código:

### 1. **Tokenización y Preprocesamiento del Texto**
   - **Tokenización**: Convierte el texto en palabras individuales (*unigrams*), o en pares de palabras (*bigrams*). Puedes usar ambos (combinados) según lo que quieras analizar.
   - **Eliminar stopwords**: Remueve palabras comunes y sin valor semántico como “el”, “de”, “que”, etc.
   - **Lematización**: Reduce las palabras a su forma base (ej., “corriendo” → “correr”).

### 2. **División del Conjunto de Datos (Train-Test Split)**
   - Antes de aplicar el esquema de ponderación, divide tus datos en un conjunto de entrenamiento y otro de prueba. Generalmente, el 80% para entrenamiento y el 20% para prueba.

### 3. **Esquema de Ponderación (Weighting Scheme)**
   Aplica uno de los siguientes métodos para convertir el texto en una representación numérica:
   - **Count Vectorizer**: Cuenta la frecuencia de aparición de cada palabra.
   - **TF-IDF (Term Frequency-Inverse Document Frequency)**: Ajusta la frecuencia de las palabras en relación a cuán comunes son en el conjunto de datos.
   - **Mutual Information**: Calcula qué palabras son más informativas para la clasificación.

### 4. **Construir el Modelo (Supervised Learning)**
   Entrena varios modelos para ver cuál se desempeña mejor. Empieza con:
   - **Modelo Base (Baseline Model)**: Entrena un modelo **Naive Bayes** como referencia inicial.
   - **SVM (Support Vector Machines)**: Un modelo efectivo para texto.
   - **XGBoost**: Un modelo basado en árboles que suele ser más potente, aunque más complejo.
   - **Deep Learning (LSTM)**: Si tienes suficiente tiempo y datos, prueba un modelo de red neuronal recurrente LSTM para capturar la secuencia de las palabras.

### 5. **Evaluación**
   Después de entrenar los modelos, evalúa su rendimiento en el conjunto de prueba usando métricas como *accuracy*, *precision*, *recall* o *F1 score*.

---

El objetivo es comparar los modelos para ver cuál ofrece los mejores resultados con tu dataset. 

¡Sí, puedes agregar un **pipeline**! Esto te permite encadenar varios pasos del preprocesamiento y el modelado de forma organizada y reproducible. El pipeline incluiría la tokenización, el esquema de ponderación, y el modelo de machine learning en una única estructura. Así, puedes aplicar todo el flujo a tu conjunto de datos de manera automática.

### ¿Qué estarás prediciendo?
Dado que en tu dataset tienes una columna de **rating**, parece que el objetivo es predecir el **rating** basado en la **review**. Esto significa que tu problema es de **regresión**, ya que estarás prediciendo un valor continuo (el rating, que va de 1 a 10, por ejemplo).

### Pasos para el **Pipeline**:

1. **Preprocesamiento de texto**:
   - Tokenización.
   - Remoción de stopwords.
   - Lematización.

2. **Esquema de Ponderación**: 
   - Usa **TF-IDF** o **CountVectorizer** para convertir las palabras a valores numéricos.

3. **Modelo**:
   - Comienza con un modelo base como **Naive Bayes** o **Regresión Lineal**.
   - Luego, prueba modelos más avanzados como **SVM**, **XGBoost** o **LSTM** si es necesario.

4. **Evaluación del Modelo**:
   - Como el objetivo es predecir ratings (regresión), usa métricas como **Mean Squared Error (MSE)** o **R^2** para evaluar el rendimiento del modelo.

### Estructura del Pipeline
Con un pipeline, todas estas etapas se ejecutarán secuencialmente sobre tus datos. Además, puedes usar **GridSearchCV** para ajustar los hiperparámetros del modelo de manera más eficiente.

---

En resumen, estarías prediciendo el **rating** de las revisiones y puedes aplicar un pipeline para automatizar el flujo completo desde el preprocesamiento hasta la predicción.